In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from openpyxl import load_workbook
import pandas as pd
import random

In [ ]:
url = "http://audiobooks.3xforum.ro/post/2228/Devon_C_Ford/"
webpage = requests.get(url, proxies = {'http' : 'http://38.54.71.67:80'})
soup = bs(webpage.content, 'html.parser')


In [ ]:
def getName(soup):

    try:
        b = soup.find_all("table", attrs={'class':'punspacer'})
        b2 = b[0].find('b').text
        name_value=b2

    except:
        name_value = ''
    return name_value

def getDescription(soup):
    try:
        des = soup.find("span", attrs={'class':'puntext'})
        des_val = des.text
    except:
        des_val = ''
    return des_val

def getCode1(soup):
    try:
        code = soup.find("pre")
        code_val = code.text
    except:
        code_val = ''
    return code_val

def getBooks(soup):
    try:
       l1 = soup.find_all("span", attrs={'class':'puntext'})[2]
       l1 = str(l1).replace("<br/>", "/n" ).split("/n")
       substring = "span" 
       for x in l1:
            filtered_list = [x for x in l1 if substring not in x]
       filtered_list = [x for x in filtered_list if x]
       substrings_to_remove = ["Series"]
       fl= [x for x in filtered_list if not any(substring in x for substring in substrings_to_remove)]
       r = "\n".join(filtered_list)
       des_val = r
    except:
        des_val = ''
    return des_val

def getCode2(soup):
    try:
        code = soup.find_all("pre")[1]
        code_val = code.text
    except:
        code_val = ''
    return code_val

def getBooksQuantity(soup):
    try:
       l1 = soup.find_all("span", attrs={'class':'puntext'})[2]
       l1 = str(l1).replace("<br/>", "/n" ).split("/n")
       substring = "span" 
       for x in l1:
            filtered_list = [x for x in l1 if substring not in x]
       filtered_list = [x for x in filtered_list if x]
        
       substrings_to_remove = ["Series", "Books", "strong", "SERIES"]
       fl= [x for x in filtered_list if not any(substring in x for substring in substrings_to_remove)]
       count_val = fl
    except:
       count_val = ''
    return len(count_val)
        

In [ ]:
getBooksQuantity(soup)

In [ ]:
with open("validproxies.txt") as f:
    proxies = f.read().split("\n")


In [ ]:
url = 'http://audiobooks.3xforum.ro/topic/2/AUDIOBOOKS/40/'

c = 0
for i in proxies:
    try:
        webpage = requests.get(url, proxies = {'http' : 'http://' + proxies[c]})
        soup = bs(webpage.content, 'html.parser')

        a = soup.find_all("tr", attrs={'class':'puntopic'})
        link_list = []
        for i in a:
            b = i.find_all('td', attrs={'class':'puncon2'})
            link_list.append(b[1].find("a").get("href"))

        d = {'Author':[], "About":[], "code":[], "Books":[], "code2":[], "BooksQuantity":[]}
        counter = 0
        for link in link_list:
            try:
                new_webpage2 = requests.get('http://audiobooks.3xforum.ro/' + link, proxies = {'http' : 'http://' + proxies[counter]})
                print("Proxy Passed " + proxies[counter])
                new_soup = bs(new_webpage2.content, 'html.parser')
                d['Author'].append(getName(new_soup))
                d['About'].append(getDescription(new_soup))
                d['code'].append(getCode1(new_soup))
                d['Books'].append(getBooks(new_soup))
                d['code2'].append(getCode2(new_soup))
                d['BooksQuantity'].append(getBooksQuantity(new_soup))
                print("Success")
            except:
                print("proxy Failed" + proxies[counter])
                counter+=1
        break
    except:
        c+=1
        print("failed")

df = pd.DataFrame(d)
df['Author'] = df['Author'].str.replace("AUDIOBOOKS / AUDIOBOOKS / ", "")


columns = {'Author':30, "About":150, "code":70, "Books":110, "code2":80, "BooksQuantity":15}
df.to_excel("Data.xlsx")
print("Success2")
# if df.loc["Author" == 'Andrea Cremer']:
#     print("Hurray!")